In [11]:
import asyncio
import sys
import logging
import boto3
from boto3.session import Session
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
from botocore.auth import SigV4Auth

from collections.abc import AsyncGenerator
from contextlib import asynccontextmanager
from datetime import timedelta
from typing import Generator

import httpx
from anyio.streams.memory import MemoryObjectReceiveStream, MemoryObjectSendStream
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest
from botocore.credentials import Credentials
from mcp.client.streamable_http import (
    GetSessionIdCallback,
    StreamableHTTPTransport,
    streamablehttp_client,
)
from mcp.shared._httpx_utils import McpHttpClientFactory, create_mcp_http_client
from mcp.shared.message import SessionMessage
from langchain_aws import ChatBedrock
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_aws import ChatBedrock
from langchain.prompts import ChatPromptTemplate
from langchain.tools import BaseTool
import json
import os
from typing import Any
from contextlib import asynccontextmanager


In [12]:
class MCPTool(BaseTool):
    name: str
    description: str
    mcp_session: Any = None
    tool_info: Any = None
    
    def __init__(self, mcp_session, tool_info, **kwargs):
        super().__init__(
            name=tool_info.name,
            description=tool_info.description,
            mcp_session=mcp_session,
            tool_info=tool_info,
            **kwargs
        )
    
    def _run(self, **kwargs) -> str:
        """동기 실행"""
        import asyncio
        try:
            return asyncio.run(self._async_run(**kwargs))
        except Exception as e:
            return f"Tool error: {str(e)}"
    
    async def _arun(self, **kwargs) -> str:
        """비동기 실행"""
        return await self._async_run(**kwargs)
    
    async def _async_run(self, **kwargs) -> str:
        """실제 MCP 도구 실행"""
        try:
            result = await self.mcp_session.call_tool(
                self.tool_info.name, 
                kwargs
            )
            return str(result)
        except Exception as e:
            return f"MCP tool error: {str(e)}"

In [13]:
class SigV4HTTPXAuth(httpx.Auth):
    """HTTPX Auth class that signs requests with AWS SigV4."""

    def __init__(
        self,
        credentials: Credentials,
        service: str,
        region: str,
    ):
        self.credentials = credentials
        self.service = service
        self.region = region
        self.signer = SigV4Auth(credentials, service, region)

    def auth_flow(
        self, request: httpx.Request
    ) -> Generator[httpx.Request, httpx.Response, None]:
        """Signs the request with SigV4 and adds the signature to the request headers."""

        # Create an AWS request
        headers = dict(request.headers)
        # Header 'connection' = 'keep-alive' is not used in calculating the request
        # signature on the server-side, and results in a signature mismatch if included
        headers.pop("connection", None)  # Remove if present, ignore if not

        aws_request = AWSRequest(
            method=request.method,
            url=str(request.url),
            data=request.content,
            headers=headers,
        )

        # Sign the request with SigV4
        self.signer.add_auth(aws_request)

        # Add the signature header to the original request
        request.headers.update(dict(aws_request.headers))

        yield request

In [14]:
class StreamableHTTPTransportWithSigV4(StreamableHTTPTransport):
    """
    Streamable HTTP client transport with AWS SigV4 signing support.

    This transport enables communication with MCP servers that authenticate using AWS IAM,
    such as servers behind a Lambda function URL or API Gateway.
    """

    def __init__(
        self,
        url: str,
        credentials: Credentials,
        service: str,
        region: str,
        headers: dict[str, str] | None = None,
        timeout: float | timedelta = 30,
        sse_read_timeout: float | timedelta = 60 * 5,
    ) -> None:
        """Initialize the StreamableHTTP transport with SigV4 signing.

        Args:
            url: The endpoint URL.
            credentials: AWS credentials for signing.
            service: AWS service name (e.g., 'lambda').
            region: AWS region (e.g., 'us-east-1').
            headers: Optional headers to include in requests.
            timeout: HTTP timeout for regular operations.
            sse_read_timeout: Timeout for SSE read operations.
        """
        # Initialize parent class with SigV4 auth handler
        super().__init__(
            url=url,
            headers=headers,
            timeout=timeout,
            sse_read_timeout=sse_read_timeout,
            auth=SigV4HTTPXAuth(credentials, service, region),
        )

        self.credentials = credentials
        self.service = service
        self.region = region

In [15]:

@asynccontextmanager
async def streamablehttp_client_with_sigv4(
    url: str,
    credentials: Credentials,
    service: str,
    region: str,
    headers: dict[str, str] | None = None,
    timeout: float | timedelta = 30,
    sse_read_timeout: float | timedelta = 60 * 5,
    terminate_on_close: bool = True,
    httpx_client_factory: McpHttpClientFactory = create_mcp_http_client,
) -> AsyncGenerator[
    tuple[
        MemoryObjectReceiveStream[SessionMessage | Exception],
        MemoryObjectSendStream[SessionMessage],
        GetSessionIdCallback,
    ],
    None,
]:
    """
    Client transport for Streamable HTTP with SigV4 auth.

    This transport enables communication with MCP servers that authenticate using AWS IAM,
    such as servers behind a Lambda function URL or API Gateway.

    Yields:
        Tuple containing:
            - read_stream: Stream for reading messages from the server
            - write_stream: Stream for sending messages to the server
            - get_session_id_callback: Function to retrieve the current session ID
    """

    async with streamablehttp_client(
        url=url,
        headers=headers,
        timeout=timeout,
        sse_read_timeout=sse_read_timeout,
        terminate_on_close=terminate_on_close,
        httpx_client_factory=httpx_client_factory,
        auth=SigV4HTTPXAuth(credentials, service, region),
    ) as result:
        yield result

In [16]:
def create_streamable_http_transport_sigv4(mcp_url: str, service_name: str, region: str):
    session = boto3.Session()
    credentials = session.get_credentials()
    return streamablehttp_client_with_sigv4(
        url=mcp_url,
        credentials=credentials,
        service=service_name,
        region=region,
    )

In [17]:
async def llm_mcp_handler(mcp_session, region, query):
    try:
        # Get available tools
        # Get available tools
        mcp_tools = await mcp_session.list_tools()
        
        # 디버깅: 도구 구조 확인
        print(f"MCP Tools type: {type(mcp_tools)}")
        print(f"MCP Tools content: {mcp_tools}")
        
        if hasattr(mcp_tools, 'tools'):
            # tools 속성이 있는 경우
            tools_list = mcp_tools.tools
        elif isinstance(mcp_tools, (list, tuple)):
            # 리스트나 튜플인 경우
            tools_list = mcp_tools
        else:
            # 다른 구조인 경우
            tools_list = []
        
        print(f"Tools list: {tools_list}")
        
        # LangChain 도구로 변환
        langchain_tools = []
        for tool_info in tools_list:
            print(f"Tool info: {tool_info}, type: {type(tool_info)}")
            
            # tool_info가 딕셔너리인 경우
            if isinstance(tool_info, dict):
                name = tool_info.get('name', 'unknown')
                description = tool_info.get('description', 'No description')
            else:
                # 객체인 경우
                name = getattr(tool_info, 'name', 'unknown')
                description = getattr(tool_info, 'description', 'No description')
            
            # 간단한 도구 객체 생성
            class SimpleTool:
                def __init__(self, name, description):
                    self.name = name
                    self.description = description
            
        simple_tool = SimpleTool(name, description)
        langchain_tools.append(MCPTool(mcp_session, simple_tool))
        
        # Initialize LLM
        bedrock_client = boto3.client('bedrock-runtime', region_name=region)
        
        llm = ChatBedrock(
            client=bedrock_client,
            model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # 안정적인 모델로 변경
            model_kwargs={"max_tokens": 1000, "temperature": 0}
        )
        
        # Create prompt
        prompt = ChatPromptTemplate.from_messages([
            ("system", "MCP 도구를 사용하세요."),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}")
        ])
        
        agent = create_tool_calling_agent(llm, langchain_tools, prompt)
        agent_executor = AgentExecutor(
            agent=agent, 
            tools=langchain_tools,
            verbose=True,
            handle_parsing_errors=True,  # 파싱 에러 처리
            max_iterations=3  # 최대 반복 제한
        )

        # Agent 실행
        result = await agent_executor.ainvoke({
            "input": query
        })
        return f"🤖 Agent response: {result['output']}"
        
    except Exception as e:
        return f"❌ Handler Error: {str(e)}"

In [18]:
async def extract_text(payload):
    try:
        yield {"type": "status", "message": "🚀 Initializing LLM..."}
        
        boto_session = Session()
        region = boto_session.region_name or 'us-west-2'
        ssm_client = boto3.client("ssm", region_name=region)
        
        agent_arn_response = ssm_client.get_parameter(Name="/mcp_server/runtime_iam/agent_arn")
        agent_arn = agent_arn_response["Parameter"]["Value"]
        
        encoded_arn = agent_arn.replace(":", "%3A").replace("/", "%2F")
        mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
        
        yield {"type": "status", "message": "✅ Connecting to MCP..."}
        
        stderr_backup = sys.stderr
        sys.stderr = open(os.devnull, 'w')
        
        try:
            async with create_streamable_http_transport_sigv4(
                mcp_url=mcp_url, service_name="bedrock-agentcore", region=region
            ) as (read_stream, write_stream, _):
                async with ClientSession(read_stream, write_stream) as mcp_session:
                    await mcp_session.initialize()
                    
                    yield {"type": "status", "message": "✅ Processing..."}
                    
                    response = await llm_mcp_handler(mcp_session, region, payload["input_data"])
                    yield {"type": "stream", "content": response}
        finally:
            sys.stderr.close()
            sys.stderr = stderr_backup
                
    except Exception as e:
        yield {"type": "error", "message": str(e)}

In [19]:
async def run_in_jupyter():
    payload = {"input_data": "11234*778은?"}
    
    async for result in extract_text(payload):
        if result["type"] == "status":
            print(f"Status: {result['message']}")
        elif result["type"] == "stream":
            print(f"Response: {result['content']}")
        elif result["type"] == "error":
            print(f"Error: {result['message']}")



In [20]:
await run_in_jupyter()


Status: 🚀 Initializing LLM...
Status: ✅ Connecting to MCP...
Status: ✅ Processing...
MCP Tools type: <class 'mcp.types.ListToolsResult'>
MCP Tools content: meta=None nextCursor=None tools=[Tool(name='add_numbers', title=None, description='Add two numbers together', inputSchema={'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'add_numbersArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'integer'}}, 'required': ['result'], 'title': 'add_numbersOutput', 'type': 'object'}, annotations=None, meta=None), Tool(name='multiply_numbers', title=None, description='Multiply two numbers together', inputSchema={'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply_numbersArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'integer'}}, 'required